사용자가 입력한 주제에 대해 인터넷 검색을 수행하고, 검색 결과를 분석하여 해당 주제에 대한 리포트를 작성하는 챗봇을 개발합니다.

## 디자인 요구사항
- 사용자와 대화형 인터페이스로 시작해야 합니다 (Chatbot 스타일).
- 사용자가 입력한 주제가 너무 모호한 경우, 추가적인 Clarifying 질문을 해야 합니다.
- 주제가 충분히 구체적이면 웹 검색을 통해 정보를 수집하고 리포트를 작성합니다.
- 인터넷 검색은 충분한 노력을 기울여 신뢰할 수 있는 결과를 포함해야 합니다.
- 검색 키워드, 참고한 웹사이트(URL), 요약, 리포트 본문을 포함하도록 합니다.

## 실습 목표
- GPT-4o의 웹 검색 모델을 활용한 실시간 정보 수집
- 주제 명확성 판단 및 대화 흐름 설계
- 구조화된 리포트 생성 자동화
- 실제 사용 가능한 리서치 도우미 형태의 챗봇 구현

## 개발 지침
- OpenAI API의 gpt-4o 및 gpt-4o-search-preview 모델을 사용하세요.
- 사용자 입력에 따라 조건 분기 처리(clarification vs. research)를 구현하세요.
- 리포트 결과에는 검색 키워드, 출처 URL, 요약, 본문을 반드시 포함해야 합니다.
- 에러 처리 및 예외 상황(검색 실패 등)에 대한 대응도 고려하세요.

## 평가 기준
- Design Requirements 충족 여부
- 사용자 입력에 대한 정확한 흐름 제어
- 실제 검색 결과에 기반한 신뢰성 있는 리포트 생성 여부
- 코드 완성도
- UX 관점에서의 자연스러운 대화 흐름 구현



In [9]:
# 필요한 라이브러리 설치 및 import
import json
import requests
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
import re
from datetime import datetime
import time
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

ROOT_DIR = Path.cwd()
sys.path.append(str(ROOT_DIR))

ENV_PATH = ROOT_DIR / ".env"

if ENV_PATH.exists():
    load_dotenv(ENV_PATH)
    print(f".env 파일을 로드했습니다: {ENV_PATH}")
else:
    # .env 파일이 없으면 기본값 사용
    print(".env 파일이 없습니다. 기본 설정값을 사용합니다.")

# OpenAI 클라이언트 초기화
GPT_MODEL = "gpt-4o"
SEARCH_MODEL = "gpt-4o-search-preview"

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key) # OR 직접 입력

print("✅ 라이브러리 로드 완료")

.env 파일을 로드했습니다: /Users/tykim/Desktop/work/SNU_bigdata_fintech_2025/ai-application-assignment/.env
✅ 라이브러리 로드 완료


In [10]:
# 유틸리티 함수들
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, model=GPT_MODEL):
    """Chat Completions API 호출"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
        )
        return response
    except Exception as e:
        print(f"❌ ChatCompletion 응답 생성 실패: {e}")
        return e

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def web_search_request(query, model=SEARCH_MODEL):
    """웹 검색 API 호출"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": query}],
        )
        return response
    except Exception as e:
        print(f"❌ 웹 검색 실패: {e}")
        return e

def pretty_print_conversation(messages):
    """대화 내용을 컬러로 출력"""
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
    }
    
    for message in messages:
        if message["role"] in role_to_color:
            print(colored(f"{message['role']}: {message['content']}\n", role_to_color[message["role"]]))

print("✅ 유틸리티 함수 로드 완료")


✅ 유틸리티 함수 로드 완료


In [11]:
# 주제 명확성 판단 및 추가 질문 기능
class TopicClarifier:
    def __init__(self):
        # 기본 질문 템플릿 (더 유연한 접근)
        self.base_question_templates = {
            "vague_topic": [
                "어떤 특정한 측면이나 관점에서 {topic}에 대해 알고 싶으신가요?",
                "{topic}의 어떤 부분에 가장 관심이 있으신가요?",
                "{topic}에 대해 구체적으로 어떤 정보를 원하시나요?"
            ],
            "missing_context": [
                "어떤 맥락이나 상황에서 {topic}에 대해 알고 싶으신가요?",
                "{topic}와 관련하여 특별히 관심 있는 영역이 있나요?",
                "{topic}에 대해 어떤 수준의 정보를 원하시나요? (기초적인 개념, 최신 동향, 전문적인 분석 등)"
            ],
            "too_broad": [
                "{topic}은 매우 광범위한 주제입니다. 어떤 특정 영역에 집중하고 싶으신가요?",
                "{topic} 중에서 가장 관심 있는 세부 주제는 무엇인가요?",
                "{topic}에 대해 어떤 관점에서 접근하고 싶으신가요?"
            ]
        }
        
        # 분야별 특화 질문 패턴
        self.domain_specific_patterns = {
            "academic": ["연구 동향", "이론적 배경", "실험 결과", "학술적 관점"],
            "technology": ["기술적 특징", "최신 개발", "적용 분야", "기술적 한계"],
            "business": ["시장 동향", "경제적 영향", "비즈니스 모델", "경쟁사 분석"],
            "entertainment": ["최신 소식", "인기 트렌드", "팬 반응", "미디어 보도"],
            "daily_life": ["실생활 적용", "일상적 영향", "실용적 정보", "개인적 관점"],
            "science": ["과학적 원리", "실험 결과", "연구 동향", "과학적 증거"],
            "politics": ["정치적 영향", "정책 변화", "국제 관계", "정치적 관점"],
            "health": ["건강 영향", "의학적 관점", "예방법", "치료법"]
        }
    
    def analyze_topic_clarity(self, topic):
        """주제의 명확성을 분석"""
        messages = [
            {
                "role": "system", 
                "content": """당신은 주제의 명확성을 분석하는 전문 AI입니다. 
                사용자가 입력한 주제를 다양한 관점에서 분석하여 명확성을 판단하고, 
                모호하거나 불완전한 경우 적절한 명확화 질문을 생성해주세요.
                
                분석 시 고려사항:
                1. 주제의 구체성과 명확성
                2. 주제가 속한 분야 (학문, 기술, 비즈니스, 연예, 일상생활, 과학, 정치, 건강 등)
                3. 필요한 맥락이나 배경 정보
                4. 사용자가 원하는 정보의 수준과 깊이
                5. 시간적 범위 (과거, 현재, 미래)
                6. 지리적 범위 (국내, 국제, 특정 지역)
                
                분석 결과를 다음 JSON 형식으로 반환해주세요:
                {
                    "is_clear": true/false,
                    "clarity_type": "clear" | "vague_topic" | "missing_context" | "too_broad" | "domain_ambiguous",
                    "detected_domain": "academic" | "technology" | "business" | "entertainment" | "daily_life" | "science" | "politics" | "health" | "unknown",
                    "suggested_questions": ["질문1", "질문2", "질문3"],
                    "reasoning": "판단 근거",
                    "confidence": 0.0-1.0
                }"""
            },
            {
                "role": "user",
                "content": f"다음 주제의 명확성을 종합적으로 분석해주세요: '{topic}'"
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            return {"is_clear": False, "clarity_type": "error", "reasoning": str(response), "confidence": 0.0}
        
        try:
            content = response.choices[0].message.content
            # JSON 부분만 추출
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                result = json.loads(json_match.group())
                # 기본값 설정
                result.setdefault("confidence", 0.8)
                result.setdefault("detected_domain", "unknown")
                return result
            else:
                return {"is_clear": False, "clarity_type": "error", "reasoning": "JSON 파싱 실패", "confidence": 0.0}
        except Exception as e:
            return {"is_clear": False, "clarity_type": "error", "reasoning": f"파싱 오류: {e}", "confidence": 0.0}
    
    def generate_clarification_questions(self, topic, clarity_analysis):
        """명확한 질문 생성"""
        clarity_type = clarity_analysis.get("clarity_type", "vague_topic")
        detected_domain = clarity_analysis.get("detected_domain", "unknown")
        confidence = clarity_analysis.get("confidence", 0.5)
        
        # AI가 제안한 질문이 있으면 우선 사용
        if "suggested_questions" in clarity_analysis and clarity_analysis["suggested_questions"]:
            return clarity_analysis["suggested_questions"]
        
        # 기본 템플릿 기반 질문 생성
        if clarity_type in self.base_question_templates:
            base_questions = self.base_question_templates[clarity_type]
        else:
            base_questions = self.base_question_templates["vague_topic"]
        
        questions = [q.format(topic=topic) for q in base_questions]
        
        # 분야별 특화 질문 추가
        if detected_domain != "unknown" and detected_domain in self.domain_specific_patterns:
            domain_patterns = self.domain_specific_patterns[detected_domain]
            domain_question = f"{topic}에 대해 {', '.join(domain_patterns[:2])} 중 어떤 측면이 가장 궁금하신가요?"
            questions.append(domain_question)
        
        # 신뢰도가 낮으면 추가 질문
        if confidence < 0.6:
            questions.append(f"{topic}에 대해 어떤 관점에서 접근하고 싶으신가요?")
        
        return questions[:4]  # 최대 4개 질문으로 제한

print("✅ 주제 명확성 판단 클래스 로드 완료")


✅ 주제 명확성 판단 클래스 로드 완료


In [12]:
# 웹 검색 및 정보 수집 기능
class WebSearchEngine:
    def __init__(self):
        self.search_history = []
        self.search_results = []
    
    def search_web(self, query, max_searches=3):
        """웹에서 정보 검색 (AI 기반 동적 쿼리 생성)"""
        print(f"🔍 '{query}'에 대해 웹 검색 중...")
        
        # AI가 동적으로 검색 쿼리 생성
        search_queries = self._generate_dynamic_search_queries(query, max_searches)
        
        search_results = []
        
        # 생성된 쿼리들로 검색 수행
        for i, search_query in enumerate(search_queries):
            try:
                print(f"  📊 검색 {i+1}/{len(search_queries)}: {search_query}")
                
                response = web_search_request(search_query)
                
                if isinstance(response, Exception):
                    print(f"  ❌ 검색 {i+1} 실패: {response}")
                    continue
                
                content = response.choices[0].message.content
                
                # 검색 결과 파싱 및 저장
                search_result = {
                    "query": search_query,
                    "content": content,
                    "timestamp": datetime.now().isoformat(),
                    "search_number": i + 1
                }
                
                search_results.append(search_result)
                print(f"  ✅ 검색 {i+1} 완료")
                
                # 검색 간 간격 (API 제한 고려)
                time.sleep(1)
                
            except Exception as e:
                print(f"  ❌ 검색 {i+1} 중 오류 발생: {e}")
                continue
        
        self.search_results.extend(search_results)
        return search_results
    
    def _generate_dynamic_search_queries(self, topic, max_searches=3):
        """AI가 주제에 맞는 동적 검색 쿼리 생성"""
        print(f"  🤖 '{topic}'에 대한 최적화된 검색 쿼리를 생성 중...")
        
        messages = [
            {
                "role": "system",
                "content": f"""당신은 웹 검색 전문가입니다. 
                주어진 주제에 대해 효과적인 검색 쿼리를 생성해주세요.
                
                요구사항:
                1. 주제의 특성과 분야를 고려하여 다양한 관점에서 검색
                2. 최신 정보, 전문적 분석, 실용적 정보를 모두 포함
                3. 검색 결과의 다양성과 신뢰성을 높이는 쿼리 생성
                4. 각 쿼리는 서로 다른 측면을 다루어야 함
                5. 한국어와 영어를 적절히 활용
                
                생성할 쿼리 수: {max_searches}개
                
                결과를 다음 JSON 형식으로 반환해주세요:
                {{
                    "queries": ["쿼리1", "쿼리2", "쿼리3"],
                    "reasoning": "쿼리 생성 전략 설명"
                }}"""
            },
            {
                "role": "user",
                "content": f"다음 주제에 대한 검색 쿼리를 생성해주세요: '{topic}'"
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            print(f"  ⚠️ AI 쿼리 생성 실패, 기본 쿼리 사용: {response}")
            return self._get_fallback_queries(topic, max_searches)
        
        try:
            content = response.choices[0].message.content
            # JSON 부분만 추출
            json_match = re.search(r'\{.*\}', content, re.DOTALL)
            if json_match:
                result = json.loads(json_match.group())
                queries = result.get("queries", [])
                reasoning = result.get("reasoning", "AI가 동적으로 생성")
                
                print(f"  💡 생성 전략: {reasoning}")
                
                # 쿼리 수 조정
                if len(queries) > max_searches:
                    queries = queries[:max_searches]
                elif len(queries) < max_searches:
                    # 부족한 쿼리는 기본 패턴으로 보완
                    fallback_queries = self._get_fallback_queries(topic, max_searches - len(queries))
                    queries.extend(fallback_queries)
                
                return queries
            else:
                print(f"  ⚠️ JSON 파싱 실패, 기본 쿼리 사용")
                return self._get_fallback_queries(topic, max_searches)
                
        except Exception as e:
            print(f"  ⚠️ 쿼리 생성 중 오류: {e}, 기본 쿼리 사용")
            return self._get_fallback_queries(topic, max_searches)
    
    def _get_fallback_queries(self, topic, max_searches):
        """AI 쿼리 생성 실패 시 사용할 기본 쿼리들"""
        fallback_patterns = [
            topic,  # 원본 주제
            f"{topic} 최신 동향",
            f"{topic} 상세 분석"
        ]
        return fallback_patterns[:max_searches]
    
    def extract_key_information(self, search_results):
        """검색 결과에서 핵심 정보 추출"""
        print("검색 결과에서 핵심 정보를 추출하고 있습니다...")
        
        combined_content = "\n\n".join([result["content"] for result in search_results])
        
        messages = [
            {
                "role": "system",
                "content": """당신은 웹 검색 결과를 분석하여 핵심 정보를 추출하는 AI입니다.
                다음 작업을 수행해주세요:
                1. 검색 결과에서 중복되는 정보 제거
                2. 가장 중요하고 신뢰할 수 있는 정보 선별
                3. 정보의 출처와 신뢰성 평가
                4. 핵심 내용을 구조화하여 정리
                
                결과를 다음 형식으로 제공해주세요:
                - 주요 발견사항
                - 신뢰할 수 있는 출처
                - 핵심 통계나 데이터
                - 최신 동향이나 트렌드"""
            },
            {
                "role": "user",
                "content": f"다음 검색 결과를 분석해주세요:\n\n{combined_content}"
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            return f"정보 추출 중 오류 발생: {response}"
        
        return response.choices[0].message.content
    
    def get_search_summary(self):
        """검색 요약 정보 반환"""
        if not self.search_results:
            return "검색 결과가 없습니다."
        
        summary = {
            "total_searches": len(self.search_results),
            "queries_used": [result["query"] for result in self.search_results],
            "search_timestamp": self.search_results[0]["timestamp"] if self.search_results else None
        }
        
        return summary

print("✅ 웹 검색 엔진 클래스 로드 완료")


✅ 웹 검색 엔진 클래스 로드 완료


In [13]:
# 리포트 생성 기능
class ReportGenerator:
    def __init__(self):
        self.report_templates = {
            "basic": {
                "title": "리포트 제목",
                "sections": ["요약", "주요 내용", "결론"],
                "format": "기본 리포트 형식"
            },
            "detailed": {
                "title": "상세 분석 리포트",
                "sections": ["요약", "배경", "주요 내용", "분석", "결론", "참고 자료"],
                "format": "상세 분석 리포트 형식"
            }
        }
    
    def generate_report(self, topic, search_results, key_information):
        """검색 결과를 바탕으로 리포트 생성"""
        print(f"📝 '{topic}'에 대한 리포트를 생성하고 있습니다...")
        
        # 검색 키워드 추출
        search_keywords = [result["query"] for result in search_results]
        
        # 참고 웹사이트 URL 추출 (검색 결과에서)
        reference_urls = self._extract_urls(search_results)
        
        # 리포트 본문 생성
        report_content = self._create_report_content(topic, search_results, key_information)
        
        # 최종 리포트 구조화
        final_report = {
            "report_title": f"{topic}에 대한 연구 리포트",
            "generated_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "search_keywords": search_keywords,
            "reference_urls": reference_urls,
            "summary": self._create_summary(key_information),
            "report_body": report_content,
            "conclusion": self._create_conclusion(topic, key_information)
        }
        
        return final_report
    
    def _extract_urls(self, search_results):
        """검색 결과에서 URL 추출 (견고한 파싱 및 검증)"""
        urls = []
        seen = set()
        
        # 패턴들: 마크다운 링크, 일반 URL, 도메인 형태(스킴 없는 경우)
        md_link_pattern = r"\[[^\]]*\]\((https?://[^\s)]+)\)"
        raw_url_pattern = r"(https?://[A-Za-z0-9\-._~:/?#\[\]@!$&'()*+,;=%]+)"
        domain_like_pattern = r"\b((?:www\.)?[A-Za-z0-9\-]+(?:\.[A-Za-z0-9\-]+)+(?:/[\w\-._~:/?#\[\]@!$&'()*+,;=%]*)?)\b"
        
        def normalize_url(u: str) -> str:
            # 좌우 공백 및 문장부호 제거
            u = u.strip().strip(").,;:'\"!?)")
            # 괄호 짝 안맞는 경우 끝 괄호 제거
            if u.endswith(')') and '(' not in u:
                u = u[:-1]
            if not u.startswith("http://") and not u.startswith("https://"):
                u = "https://" + u
            return u
        
        def has_valid_tld(u: str) -> bool:
            m = re.match(r"https?://([^/]+)", u)
            if not m:
                return False
            host = m.group(1)
            # 아이디/패스워드가 포함된 경우 제거
            if '@' in host:
                host = host.split('@', 1)[-1]
            # 포트 제거
            if ':' in host:
                host = host.split(':', 1)[0]
            # 최소 하나의 점과 영문 TLD 필요
            if '.' not in host:
                return False
            tld = host.split('.')[-1]
            return bool(re.fullmatch(r"[A-Za-z]{2,24}", tld))
        
        for result in search_results:
            content = result.get("content", "") or ""
            candidates = []
            # 1) 마크다운 링크에서 URL 추출
            candidates += re.findall(md_link_pattern, content)
            # 2) 일반 URL 추출
            candidates += re.findall(raw_url_pattern, content)
            # 3) 도메인 형태 추출 (스킴 보강 필요)
            candidates += re.findall(domain_like_pattern, content)
            
            for cand in candidates:
                url = normalize_url(cand)
                if has_valid_tld(url) and url not in seen:
                    seen.add(url)
                    urls.append(url)
                if len(urls) >= 10:
                    break
        
        return urls[:5]  # 상위 5개만 반환
    
    def _create_summary(self, key_information):
        """요약 생성"""
        messages = [
            {
                "role": "system",
                "content": "당신은 정보를 요약하는 전문가입니다. 주어진 정보를 바탕으로 핵심 내용을 3-4문장으로 간결하게 요약해주세요."
            },
            {
                "role": "user",
                "content": f"다음 정보를 요약해주세요:\n\n{key_information}"
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            return f"요약 생성 중 오류 발생: {response}"
        
        return response.choices[0].message.content
    
    def _create_report_content(self, topic, search_results, key_information):
        """리포트 본문 생성"""
        combined_search_content = "\n\n".join([result["content"] for result in search_results])
        
        messages = [
            {
                "role": "system",
                "content": f"""당신은 전문적인 리포트 작가입니다. 
                주어진 주제와 검색 결과를 바탕으로 구조화된 리포트 본문을 작성해주세요.
                
                리포트는 다음 구조를 따라야 합니다:
                1. 서론 - 주제에 대한 개요
                2. 본론 - 주요 내용과 분석
                3. 세부 분석 - 구체적인 정보와 데이터
                4. 시사점 - 중요성과 영향
                
                리포트는 객관적이고 신뢰할 수 있는 정보를 바탕으로 작성하며,
                출처를 명시하고 최신 정보를 포함해야 합니다."""
            },
            {
                "role": "user",
                "content": f"""주제: {topic}
                
                검색 결과:
                {combined_search_content}
                
                핵심 정보:
                {key_information}
                
                위 정보를 바탕으로 상세한 리포트 본문을 작성해주세요."""
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            return f"리포트 본문 생성 중 오류 발생: {response}"
        
        return response.choices[0].message.content
    
    def _create_conclusion(self, topic, key_information):
        """결론 생성"""
        messages = [
            {
                "role": "system",
                "content": "당신은 리포트의 결론을 작성하는 전문가입니다. 주어진 정보를 바탕으로 주제에 대한 종합적인 결론과 향후 전망을 제시해주세요."
            },
            {
                "role": "user",
                "content": f"""주제: {topic}
                
                핵심 정보:
                {key_information}
                
                위 정보를 바탕으로 결론을 작성해주세요."""
            }
        ]
        
        response = chat_completion_request(messages)
        if isinstance(response, Exception):
            return f"결론 생성 중 오류 발생: {response}"
        
        return response.choices[0].message.content
    
    def format_report(self, report):
        """리포트를 보기 좋게 포맷팅"""
        formatted_report = f"""
{'='*60}
📊 {report['report_title']}
{'='*60}

📅 생성일시: {report['generated_at']}

🔍 검색 키워드:
{chr(10).join([f'  • {keyword}' for keyword in report['search_keywords']])}

📚 참고 웹사이트:
{chr(10).join([f'  • {url}' for url in report['reference_urls']]) if report['reference_urls'] else '  • 참고 URL이 없습니다.'}

📋 요약:
{report['summary']}

📝 리포트 본문:
{report['report_body']}

🎯 결론:
{report['conclusion']}

{'='*60}
"""
        return formatted_report

print("✅ 리포트 생성기 클래스 로드 완료")


✅ 리포트 생성기 클래스 로드 완료


In [14]:
# 메인 챗봇 클래스
class ResearchChatbot:
    def __init__(self):
        self.topic_clarifier = TopicClarifier()
        self.web_search_engine = WebSearchEngine()
        self.report_generator = ReportGenerator()
        self.conversation_history = []

    def start_conversation(self):
        """대화형 인터페이스 시작"""
        print("🤖 안녕하세요! 저는 인터넷 검색을 통해 리포트를 작성하는 AI 어시스턴트입니다.")
        print("🤖 어떤 주제에 대해 리포트를 작성하고 싶으신가요?")
        print("❌ 종료하려면 'quit', 'exit', '종료'를 입력하세요.\n")

        while True:
            try:
                user_input = input("👤 사용자: ").strip()

                if user_input.lower() in ['quit', 'exit', '종료']:
                    print("🤖 어시스턴트: 대화를 종료합니다. 좋은 하루 되세요! 👋")
                    break

                if not user_input:
                    print("🤖 어시스턴트: 주제를 입력해주세요.\n")
                    continue

                print(f"👤 사용자: {user_input}\n")

                print(f"🤖 어시스턴트: '{user_input}'에 대해 분석해보겠습니다...\n")

                # 주제 처리
                is_completed = self.process_topic(user_input)

                # 리포트가 완료되면 자동 종료
                if is_completed:
                    break

            except KeyboardInterrupt:
                print("\n🤖 어시스턴트: 대화를 종료합니다. 좋은 하루 되세요! 👋")
                break
            except Exception as e:
                print(f"🤖 어시스턴트: 오류가 발생했습니다: {e}")
                print("다시 시도해주세요.\n")

    def process_topic(self, topic):
        """주제 처리 메인 로직"""
        try:
            # 1. 주제 명확성 분석
            print("🔍 주제의 명확성을 분석하고 있습니다... \n")
            clarity_analysis = self.topic_clarifier.analyze_topic_clarity(topic)

            if clarity_analysis["is_clear"]:
                print("✅ 주제가 충분히 명확합니다. 리포트 작성을 시작합니다.\n")
                return self.generate_research_report(topic)
            else:
                # print(f"⚠️ 주제가 모호합니다: {clarity_analysis['reasoning']}\n")
                return self.handle_unclear_topic(topic, clarity_analysis)

        except Exception as e:
            print(f"❌ 주제 처리 중 오류 발생: {e}")
            return False

    def handle_unclear_topic(self, topic, clarity_analysis):
        """모호한 주제 처리 (개선된 버전)"""
        detected_domain = clarity_analysis.get("detected_domain", "unknown")
        confidence = clarity_analysis.get("confidence", 0.5)

        # 개선된 명확화 질문 생성
        clarification_questions = self.topic_clarifier.generate_clarification_questions(topic, clarity_analysis)

        print("🤖 어시스턴트: 주제를 더 구체적으로 명확히 하기 위해 몇 가지 질문이 있습니다.\n")

        # 분야 정보 표시
        if detected_domain != "unknown":
            domain_names = {
                "academic": "학술/연구",
                "technology": "기술",
                "business": "비즈니스",
                "entertainment": "연예/엔터테인먼트",
                "daily_life": "일상생활",
                "science": "과학",
                "politics": "정치",
                "health": "건강/의료"
            }
            print(f"📋 감지된 분야: {domain_names.get(detected_domain, detected_domain)}")
            print(f"🎯 신뢰도: {confidence:.1%}\n")

        for i, question in enumerate(clarification_questions, 1):
            print(f"{i}. {question}")

        print("\n위 질문들에 대한 답변을 통해 주제를 구체화해주세요.\n")
        print("또는 더 구체적인 주제를 직접 입력해주세요.\n")

        clarification_input = input("\n👤 사용자 (명확한 답변): ").strip()

        print(f"👤 사용자: {clarification_input}\n")

        if clarification_input:
            refined_topic = f"{topic} - {clarification_input}"
            print(f"\n🤖 어시스턴트: '{refined_topic}'로 주제를 구체화했습니다. 리포트 작성을 시작합니다.\n")
            return self.generate_research_report(refined_topic)
        else:
            print("🤖 어시스턴트: 명확한 주제를 입력해주세요.\n")
            return False

    def generate_research_report(self, topic):
        """연구 리포트 생성"""
        try:
            # 2. 웹 검색 수행
            search_results = self.web_search_engine.search_web(topic, max_searches=3)

            if not search_results:
                print("❌ 웹 검색 결과를 가져올 수 없습니다. 다시 시도해주세요.")
                return False

            # 3. 핵심 정보 추출
            key_information = self.web_search_engine.extract_key_information(search_results)

            # 4. 리포트 생성
            report = self.report_generator.generate_report(topic, search_results, key_information)

            # 5. 리포트 출력
            print("\n" + "="*80)
            print("📊 리포트 생성 완료!")
            print("="*80)

            formatted_report = self.report_generator.format_report(report)
            print(formatted_report)

            # 6. 대화 기록 저장
            self.conversation_history.append({
                "topic": topic,
                "timestamp": datetime.now().isoformat(),
                "report": report
            })

            print("\n🤖 어시스턴트: 리포트가 완성되었습니다!")
            print("🎉 작업이 완료되어 챗봇을 종료합니다. 좋은 하루 되세요! 👋")

            return True  # 성공적으로 완료됨을 나타내는 플래그

        except Exception as e:
            print(f"❌ 리포트 생성 중 오류 발생: {e}")
            print("다시 시도해주세요.\n")
            return False

    def get_conversation_history(self):
        """대화 기록 조회"""
        if not self.conversation_history:
            return "대화 기록이 없습니다."

        history = "📚 대화 기록:\n"
        for i, conv in enumerate(self.conversation_history, 1):
            history += f"{i}. {conv['topic']} ({conv['timestamp']})\n"

        return history

print("✅ 메인 챗봇 클래스 로드 완료")

✅ 메인 챗봇 클래스 로드 완료


In [15]:
# 챗봇 초기화 및 테스트 함수
def initialize_chatbot():
    """챗봇 초기화"""
    global chatbot
    chatbot = ResearchChatbot()
    print("🤖 연구 리포트 생성 챗봇이 초기화되었습니다!")
    return chatbot


print("✅ 초기화 함수 로드 완료")

✅ 초기화 함수 로드 완료


In [16]:
# 챗봇 실행

# 챗봇 초기화
chatbot = initialize_chatbot()

print("=" * 60)
chatbot.start_conversation()

🤖 연구 리포트 생성 챗봇이 초기화되었습니다!
🤖 안녕하세요! 저는 인터넷 검색을 통해 리포트를 작성하는 AI 어시스턴트입니다.
🤖 어떤 주제에 대해 리포트를 작성하고 싶으신가요?
❌ 종료하려면 'quit', 'exit', '종료'를 입력하세요.

👤 사용자: 한국의 kpop 아이돌

🤖 어시스턴트: '한국의 kpop 아이돌'에 대해 분석해보겠습니다...

🔍 주제의 명확성을 분석하고 있습니다... 

🤖 어시스턴트: 주제를 더 구체적으로 명확히 하기 위해 몇 가지 질문이 있습니다.

📋 감지된 분야: 연예/엔터테인먼트
🎯 신뢰도: 85.0%

1. 어떤 특정 K-pop 아이돌 그룹이나 개인을 지칭하나요?
2. K-pop 아이돌의 역사, 현재 활동, 또는 영향력에 대해 알고 싶으신가요?
3. 한국 내 K-pop 아이돌의 팬 문화나 산업 구조에 대해 알고 싶으신가요?
4. K-pop 아이돌의 연습생 시스템이나 데뷔 과정에 대해 궁금하신가요?

위 질문들에 대한 답변을 통해 주제를 구체화해주세요.

또는 더 구체적인 주제를 직접 입력해주세요.

👤 사용자: 아이돌 팬 문화와 산업 구조


🤖 어시스턴트: '한국의 kpop 아이돌 - 아이돌 팬 문화와 산업 구조'로 주제를 구체화했습니다. 리포트 작성을 시작합니다.

🔍 '한국의 kpop 아이돌 - 아이돌 팬 문화와 산업 구조'에 대해 웹 검색 중...
  🤖 '한국의 kpop 아이돌 - 아이돌 팬 문화와 산업 구조'에 대한 최적화된 검색 쿼리를 생성 중...
  💡 생성 전략: 주제의 다양성을 고려하여, 첫 번째 쿼리는 K-pop 팬 문화의 최신 트렌드를 살펴볼 수 있는 정보를 찾기 위해 설정되었습니다. 이는 최근 팬 문화의 변화와 새로운 트렌드에 관한 자료를 제공할 수 있습니다. 두 번째 쿼리는 한국의 K-pop 산업 구조에 대한 심도 있는 분석을 위해 준비되었습니다. 이런 분석은 산업 내부의 구조 및 동향을 이해하는 데 도움이 됩니다. 세 번째 쿼리는 K-pop 아이돌 팬덤이 경제에 미치는 영향에 대한 실질적 